# コンパイラ実験（まとめ）

ライブラリ（cumps.js）の読み込み

In [ ]:
require("./cumps.js");
function print(s) { console.log(s); }

実験1: CSVデータの読み込み

In [ ]:
var num = pat("[0-9]+");
var csv = sepBy(num, ",");

In [ ]:
csv("12,345,67,8");

実験2: 電卓

In [ ]:
function expr(s) { return expr(s); }
var num    = pat("[0-9]+");
var factor = oneOf(num, seq("(", expr, ")"));
var term   = sepBy(factor, oneOf(w("*"), word("/")));
var expr   = sepBy(term, oneOf(w("+"), w("-")));

function evalExpr(ts) {
    var v = ts[0];
    for (var i = 1; i < ts.length - 1; i += 2) {
        switch (ts[i]) {
          case "+" : v += ts[i + 1]; break;
          case "-" : v -= ts[i + 1]; break;
          case "*" : v *= ts[i + 1]; break;
          case "/" : v = parseInt(v / ts[i + 1]); 
        }
    }
    return v;
}

num.action = parseInt;
term.action = expr.action = evalExpr;

In [ ]:
expr("1+2*3")

実験3: 簡単なコンパイラ（式の処理）

In [ ]:
var expr   = function (s) { return expr(s); };
var id     = pat("[a-zA-Z_][a-zA-Z0-9_]*");
var num    = pat("[0-9]+");
var expr0  = oneOf(id, num, seq("(", expr, ")"));
var expr1  = sepBy(expr0, oneOf(w("*"), w("/")));
var expr2  = sepBy(expr1, oneOf(w("+"), w("-")));
var expr3  = sepBy(expr2, oneOf(w(">"), w("<")));
var expr   = sepBy(expr3, w("=="));

num.action = function (s) {
    var n = parseInt(s);
    return function () { return n; };
};

id.action = function (x) {
    return function () { return state[x]; };
};

function compileExpr(ts) {
    return function() {
        var v = ts[0]();
        for (var i = 1; i < ts.length - 1; i += 2) {
            switch (ts[i]) {
              case "+": v += ts[i + 1](); break;
              case "-": v -= ts[i + 1](); break;
              case "*": v *= ts[i + 1](); break;
              case "/": v = parseInt(v / ts[i + 1]()); break;
              case "<": v = v < ts[i + 1](); break;
              case ">": v = v > ts[i + 1](); break;
              case "==": v = v === ts[i + 1]();
            }
        }
        return v;
    };
}

expr1.action = expr2.action = expr3.action = expr.action = compileExpr;

簡単なコンパイラ（実行文の処理）

In [ ]:
function stmt(s) { return stmt(s); }
var lval       = pat("[a-zA-Z_][a-zA-Z0-9_]*");
var assignStmt = seq(lval, "=", expr, ";");
var whileStmt  = seq("while", "(", expr, ")", stmt);
var ifStmt     = seq("if", "(", expr, ")", stmt, opt(seq("else", stmt)));
var printStmt  = seq("print", expr, ";");
var program    = moreThan0(stmt);
var stmt       = oneOf(assignStmt, whileStmt, ifStmt, printStmt, seq("{", program, "}"));

assignStmt.action = function(ts) {
    var x = ts[0], evalRHS = ts[1];
    return function () { state[x] = evalRHS(); };
};

whileStmt.action = function(ts) {
    var evalC = ts[0], runStmt = ts[1];
    return function () { while (evalC()) runStmt(); };
};

ifStmt.action = function(ts) {
    var evalC = ts[0], runIf = ts[1], runElse = ts[2] ? ts[2] : function() {};
    return function() { if (evalC()) runIf(); else runElse(); };       
};

printStmt.action = function(ts) {
    var eval = ts[0];
    return function () { print(eval()); };
};

program.action = function (ts) {
    return function() { for (i = 0; i < ts.length; i++) ts[i](); }; 
};

In [ ]:
var fib ="\
n = 30;\
a = 0;\
b = 1;\
while (n > 0) {\
  print b;\
  c = b;\
  b = a + b;\
  a = c;\
  n = n - 1;\
}\
"
state = {};
(program(fib).a)();

In [ ]:
var fs = require("fs");
function run(file) {
    var src = fs.readFileSync(file, {encoding : "utf8"});
    state = {};
    var aout = program(src).a;
    aout();
}

In [ ]:
run("./samples/fib.toy");